# 7장
1. multi-inputs Model
2. callbacks

### 질문-응답모형 : 입력 : 1. 자연어 질문, 2. 답변에 필요한 정보가 있는 텍스트(예_기사)

In [1]:
from keras.models import Model
from keras import layers
from keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

text_input = Input(shape=(None,), dtype='int32', name='text')
embedded_text = layers.Embedding(text_vocabulary_size, 64)(text_input)
encoded_text = layers.LSTM(32)(embedded_text)

question_input = Input(shape=(None,), dtype='int32', name='question') 
embedded_question = layers.Embedding(question_vocabulary_size, 32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

concatenated = layers.concatenate([encoded_text, encoded_question], axis=-1)

answer = layers.Dense(answer_vocabulary_size, activation='softmax')(concatenated)

model = Model([text_input, question_input], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


In [2]:
import numpy as np
from keras.utils import to_categorical

num_samples = 1000
max_length = 100

text = np.random.randint(1, text_vocabulary_size, size=(num_samples, max_length))
question = np.random.randint(1, question_vocabulary_size, size=(num_samples, max_length))

answers = np.random.randint(0, answer_vocabulary_size, size=num_samples)
answers = to_categorical(answers)


In [3]:
model.fit([text, question], answers, epochs=10, batch_size=128)
#model.fit({'text': text, 'question': question}, answers, epochs=10, batch_size=128)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
1000/1000 [==============================] - 4s 4ms/step - loss: 6.2145 - acc: 1.0000e-03
Epoch 2/10
1000/1000 [==============================] - 2s 2ms/step - loss: 6.1981 - acc: 0.0510
Epoch 3/10
1000/1000 [==============================] - 2s 2ms/step - loss: 6.1306 - acc: 0.0060
Epoch 4/10
1000/1000 [==============================] - 2s 2ms/step - loss: 6.0472 - acc: 0.0040
Epoch 5/10
1000/1000 [==============================] - 2s 2ms/step - loss: 5.9864 - acc: 0.0100
Epoch 6/10
1000/1000 [==============================] - 2s 2ms/step - loss: 5.9314 - acc: 0.0090
Epoch 7/10
1000/1000 [==============================] - 2s 2ms/step - loss: 5.8649 - acc: 0.0090
Epoch 8/10
1000/1000 [==============================] - 2s 2ms/step - loss: 5.7937 - acc: 0.0110
Epoch 9/10
1000/1000 [==============================] - 2s 2ms/step - loss: 5.7204 - acc: 0.0180
Epoch 10/10
1000/1000 [==============================] - 2s 2ms/step - loss

In [4]:
from keras import layers
from keras import Input
from keras.models import Model

vocabulary_size = 50000
num_income_groups = 10

posts_input = Input(shape=(None, ), dtype='int32', name='posts')
embedded_posts = layers.Embedding(vocabulary_size, 256)(posts_input)
x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Dense(128, activation='relu')(x)

age_prediction = layers.Dense(1, name='age')(x)
income_prediction = layers.Dense(num_income_groups, activation='softmax', name='income')(x)
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)

model = Model(posts_input, [age_prediction, income_prediction, gender_prediction])

In [5]:
model.compile(optimizer='rmsprop', loss = {'age':'mse', 'income':'categorical_crossentropy', 'gender':'binary_crossentropy'})
#if you wanna weight more certain value
#model.compile(optimizer='rmsprop', loss = {'age':'mse', 'income':'categorical_crossentropy', 'gender':'binary_crossentropy'},
#              loss_weight = {'age':0.25, 'income':1., 'gender':10.})

In [ ]:
model.fit(posts, {'age':age_targets, 'income':income_targets, 'gender':gender_targets}, epochs=10, batch_size=64)

In [ ]:
#siamese LSTM
from keras import layers
from keras import Input
from keras.models import Model

lstm = layers.LSTM(32)
left_input = Input(shape=(None, 128))
left_output = lstm(left_input)

right_input = Input(shape=(None, 128))
right_output = lstm(right_input)

merged = layers.concatenate([left_output, right_output], axis=-1)
predictions = layers.Dense(1, activation='sigmoid')(merged)

model = Model([left_input, right_input], predictions)
model.fit([left_data, right_data], targets)

## 2. Callbacks
2.1 ModelCheckpoint and EarlyStopping


In [ ]:
import keras

callback_list= [
    keras.callbacks.EarlyStopping(monitor = 'val_acc', patience=1),
    keras.callbacks.ModelCheckpoint(fliepath='my_model.h5', monitor='val_acc', save_best_only=True)
]

model.compile(optimizer='rmsprop', loss='binary_crossentropy'. metrics=['acc'])
model.fit(x, y, epochs=10, batch_size=32, callbacks=callback_list, validation_data=(x_val, y_val))

In [ ]:
callbacks_list = [
    keras.callbacks.ReduceLRnPlateau(monitor='val_acc', factor=0.1, patience=10)
]

model.fit(x, y, epochs=10, batch_size=32, callbacks=callbacks_list, validation_data=(x_val, y_val))

In [7]:
#Customized Callbacks
import keras 
import numpy as np

class ActivationLogger(keras.callbacks.Callback):
    
    def set_model(self, model):
        self.model = model
        layer_outputs = [layer.output for layer in model.layers]
        self.activations_model = keras.models.Model(model.input, layer_outputs)
    
    def on_epoch_end(self, epoch, logs=None):
        if self.validation_data is None:
            raise RuntimeError('Requires Validation data.')
        
        validation_sample = self.validation_data[0][0:1]
        activations = self.activations_model.predict(validation_sample)
        f = open('activation_at_epoch_' + str(epoch) + '.npz', 'wb')
        np.save(f, activations)
        f.close()